In [1]:
import numpy as np
from sklearn.neural_network import MLPClassifier

# Define a simple Reader class
class Reader:
    def __init__(self, id):
        self.id = id
        self.weight = 0
        self.cost = 0
        self.avg_weight_neighbors = 0
        self.avg_cost_neighbors = 0
        self.status = 0  # Target output (0 for OFF, 1 for ACTIVE)

# Define the machine learning stage
class MachineLearningStage:
    def __init__(self, num_readers, alpha):
        self.num_readers = num_readers
        self.alpha = alpha
        self.readers = [Reader(i) for i in range(num_readers)]
        self.X = []
        self.y = []

    def generate_sample_data(self):
        np.random.seed(42)  # For reproducibility
        for reader in self.readers:
            reader.weight = np.random.randint(1, 10)
            reader.cost = reader.weight * self.alpha
            reader.avg_weight_neighbors = np.random.randint(1, 10)
            reader.avg_cost_neighbors = reader.avg_weight_neighbors * self.alpha
            reader.status = np.random.choice([0, 1])

    def collect_training_data(self):
        self.X = np.array([(reader.weight, reader.cost, reader.avg_weight_neighbors, reader.avg_cost_neighbors) for reader in self.readers])
        self.y = np.array([reader.status for reader in self.readers])

    def train_neural_network(self):
        self.nn_model = MLPClassifier(hidden_layer_sizes=(16, 16), activation='relu', solver='adam', max_iter=500, random_state=42)
        self.nn_model.fit(self.X, self.y)

    def broadcast_trained_weights(self):
        return self.nn_model

# Simulation parameters
num_readers = 10
alpha = 0.5

# Run machine learning stage
ml_stage = MachineLearningStage(num_readers, alpha)
ml_stage.generate_sample_data()
ml_stage.collect_training_data()
ml_stage.train_neural_network()
trained_model = ml_stage.broadcast_trained_weights()

print("Neural network trained and model broadcasted to each reader.")


Neural network trained and model broadcasted to each reader.


In [2]:
import numpy as np

class ApplicationReader:
    def __init__(self, id, alpha, trained_model):
        self.id = id
        self.alpha = alpha
        self.status = "LOCK"
        self.weight = np.random.randint(1, 10)
        self.cost = self.weight * self.alpha
        self.avg_weight_neighbors = np.random.randint(1, 10)
        self.avg_cost_neighbors = self.avg_weight_neighbors * self.alpha
        self.nn_model = trained_model

    def prepare_features(self):
        return np.array([self.weight, self.cost, self.avg_weight_neighbors, self.avg_cost_neighbors]).reshape(1, -1)

    def predict_activation(self):
        return self.nn_model.predict(self.prepare_features())[0]

# Initialize application readers
application_readers = [ApplicationReader(i, alpha, trained_model) for i in range(num_readers)]

# Simulate the activation decision process
for reader in application_readers:
    activation = reader.predict_activation()
    if activation == 1:
        reader.status = "ACTIVE"
    print(f"Reader {reader.id}: Weight={reader.weight}, Cost={reader.cost}, "
          f"AvgWeightNeighbors={reader.avg_weight_neighbors}, AvgCostNeighbors={reader.avg_cost_neighbors}, "
          f"Status={reader.status}")

# Check the final status of each reader
for reader in application_readers:
    print(f"Reader {reader.id} status: {reader.status}")


Reader 0: Weight=9, Cost=4.5, AvgWeightNeighbors=3, AvgCostNeighbors=1.5, Status=ACTIVE
Reader 1: Weight=5, Cost=2.5, AvgWeightNeighbors=3, AvgCostNeighbors=1.5, Status=ACTIVE
Reader 2: Weight=7, Cost=3.5, AvgWeightNeighbors=5, AvgCostNeighbors=2.5, Status=ACTIVE
Reader 3: Weight=9, Cost=4.5, AvgWeightNeighbors=7, AvgCostNeighbors=3.5, Status=LOCK
Reader 4: Weight=2, Cost=1.0, AvgWeightNeighbors=4, AvgCostNeighbors=2.0, Status=ACTIVE
Reader 5: Weight=9, Cost=4.5, AvgWeightNeighbors=2, AvgCostNeighbors=1.0, Status=ACTIVE
Reader 6: Weight=9, Cost=4.5, AvgWeightNeighbors=5, AvgCostNeighbors=2.5, Status=ACTIVE
Reader 7: Weight=2, Cost=1.0, AvgWeightNeighbors=4, AvgCostNeighbors=2.0, Status=ACTIVE
Reader 8: Weight=7, Cost=3.5, AvgWeightNeighbors=8, AvgCostNeighbors=4.0, Status=LOCK
Reader 9: Weight=3, Cost=1.5, AvgWeightNeighbors=1, AvgCostNeighbors=0.5, Status=ACTIVE
Reader 0 status: ACTIVE
Reader 1 status: ACTIVE
Reader 2 status: ACTIVE
Reader 3 status: LOCK
Reader 4 status: ACTIVE
Reader